In [4]:
import re
import sys
import os
import requests
import time
import csv
from bs4 import BeautifulSoup
from bs4.element import Comment
from urllib.parse import urlparse
from os import path
from datetime import datetime

from IPython.core.debugger import set_trace



In [51]:
class CollegeCrawl:
    gap_Insecond=5
    max_Pages=5
    headers = {'User-Agent': 'Mozilla/5.0'}



    def __init__(self,_collegename, _rooturl, 
            _prioritykeywords=['apply', 'admission', 'application', 'deadline'], 
            _url_file=None,                  
            _save_to_folder=None,
            _existingurlfile=None, #csv files that were visited in the past
            _respectrobottxt=True, 
            _headers={'User-Agent':'Mozilla/5.0'} ):
        self.college=_collegename
         
        if urlparse(_rooturl).scheme=="":
            print('URL needs to have scheme. Please try again.')
            raise Exception('URL needs to have scheme')            
            return
        
        self.save_to_folder=None
        self.college=_collegename
        self.rootUrl=_rooturl
        self.priorityKeywords=_prioritykeywords
        self.respectRobottext=_respectrobottxt #currently not doing anything
        self.allUrls=set()
        self.allRankedUrls = [] 
        self.existingurlfile=None
        self.rejectedUrls=set()
        self.base_domain = self.__getDomainFromUrl(self.rootUrl)
        self.visitedUrls=set()
        self.scheme=urlparse(self.rootUrl).scheme
        
        
        if _save_to_folder==None or path.isdir(_save_to_folder)==False:
            self.save_to_folder=os.getcwd()
        else:
            self.save_to_folder=_save_to_folder
            
        #to make it less 
        if _existingurlfile==None or path.exists(_existingurlfile)==False:            
            self.existingurlfile=path.join(self.save_to_folder,re.sub("\s+", "_", self.college.strip()+'.csv'))
        else:
            self.existingurlfile=_existingurlfile
        self.allurls={}
        self.headers=_headers
        self.files=[]
    
    def __str__(self):
        return "************Crawl {}({})************".format(self.college, self.rootUrl)
    
        
    
    def __getDomainFromUrl(self, url):
        #url = "http://www.mit.edu/"
        firstIndex = url.find('//')
        length = len(url)-1 if url.endswith('/') else len(url)      
        return url[firstIndex+2:length]
    
    def __addAllUrlsInOnePage(self, response):
        soup=BeautifulSoup(response.text, 'html.parser')
       
        for link in soup.find_all('a'):
            try:
                url=link['href']                
                parsed_uri = urlparse(url )
                if parsed_uri.netloc=='':
                    absolute_url = self.rootUrl+url    
                elif parsed_uri.netloc==self.base_domain:
                    absolute_url=url
                else:
                    continue
                #netname = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri) 
                clean=re.sub(r'[_+!@#$?\\\s]+$', '', absolute_url)
                #set_trace()
                slashIndex = clean.index("//") 
                clean=''.join(clean[0:slashIndex+1])+(''.join(clean[slashIndex+1:])).replace('//', '/')
                clean=re.sub(r'[/\s]?$','', clean)
                self.allUrls.add(clean)
                    #unique_urls.add(re.sub(r'[_+!@#$?\\/^\s]+$', '', absolute_url))                  
            except:
                print("There was an error")
                continue                 
    

    """
        get all urls starting from rootUrl
    """
    def getAllUrls(self):  
        urls=self.Load_DiscoveredUrls() 
        self.allUrls.add(self.rootUrl)
        while len(self.allUrls)>len(self.visitedUrls) and len(self.visitedUrls)<=CollegeCrawl.max_Pages:
            #get one unvisited url
            unvisited_url=(self.allUrls-self.visitedUrls).pop()
               
            # visit the unvisited url, parse the content and add all the urls to allUrls set
            time.sleep(CollegeCrawl.gap_Insecond)
            try:
                # unvisited url becomes visited
                self.visitedUrls.add(unvisited_url)
                response=requests.get(unvisited_url,None, headers=CollegeCrawl.headers)
                # check whether successful
                if response.status_code !=200: 
                    self.rejectedUrls.add(unvisited_url)  
                else:
                    contentType = response.headers.get('content-type')
                    #set_trace()
                    if 'text/html' in contentType:
                        self.__addAllUrlsInOnePage(response)
                
                unvisited=[name for name, response.status.code in urls.items() if response.status.code==0]
                self.allRankedUrls = sorted(self.visitedUrls,key=lambda url: [w in url for w in self.priorityKeywords].count(True),reverse=True)

            #except requests.exceptions.RequestException as e:
            except:
                continue
        
        
       
        
    
    """
        read one page
    """
    def read_OneUrl(self, url):
        response=requests.get(url)  
        if response.status_code==200:
            if "text/html" in response.headers.get('content-type'):
                return self.get_pagetext(response)
            else: #for example .pdf file, img file
                pass #save the file directly?
        else: 
            return [[None, None, None, None]]
    
    
    def get_pagetext(self,body):
        soup = BeautifulSoup(body.text, 'html.parser') #another type is content which is byte (like image)
        texts = soup.findAll(text=True) 
        visible_texts = filter(self.tag_visible, texts)       
    
        return [ [t.parent.name,   
             t.parent.previousSibling.name if t.parent.previousSibling!=None else None, 
             t.nextSibling.name if t.nextSibling!=None else None,
             re.sub(r'[\s+\t]',' ',t) ]  for t in visible_texts if len(t.strip())>2] 
    
    def tag_visible(self,element):
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        return True 
        
    """
        Save One Page
    """
    def save_OnePage(self, url, format='csv'):
        url=url.strip()
        content=self.read_OneUrl(url) #in format of (a,a,a,a)
     
       # if folder==None:
        folder=os.getcwd()
        if path.isdir(folder)==False:
            print('folder doesnot exist')
            return
        #if filename==None:
        filename=url.replace('.', '_dot_').replace('/', '_').replace(':', '_')+'_'+datetime.now().strftime("%m_%d_%Y_%H_%M_%S")+'.csv'
    
        fullname=path.join(folder, filename)
        with open(fullname, 'w', newline='', encoding="utf-8") as newFile:
            writer = csv.writer(newFile, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
            if format != 'csv':
               # writer = 
                pass
            writer.writerow(['url', 'parent', 'ps', 'ns', 'text'])
            for lll in content: 
                lll.insert(0, url)
                writer.writerow(lll)  
        #return fullname           
    
    """
        Save summaries
    """
    def save_Summaries(self):
        print(self)
        print("Total number of URLs visisted: ", len(self.visitedUrls))
        print("Total number of URLs rejected: ", len(self.rejectedUrls))
        print("URLs visited (ranked with priority words):\n",self.allRankedUrls)
    
    """
        Ranks the urls based on the priority words
    """


    
    def setMaxpage(self, maxPages):
        CollegeCrawl.max_Pages = maxPages
        
    def setGap(self, gap):
        CollegeCrawl.gap_Insecond = gap
    
    
    def crawl(self):
        self.getAllUrls()
        for url in self.allRankedUrls:
            self.save_OnePage(url)
            
        self.save_Summaries()
            
        
        
    
    def Load_DiscoveredUrls(self, delimiter=',', hasHeader=False, header_names=['url', 'status_code']):   
        if self.existingurlfile==None:
            return {}
        else:
            if path.exists(self.existingurlfile) and re.sub('\s+', '_', self.college) in ntpath.basename(self.existingurlfile):
                df_urls=pd.read_csv(self.existingurlfile,  delimiter=delimiter).iloc[:, 0:2]   
                header_row=list(df_urls.columns) 
            
                if re.match('^http', header_row[0]):
                    df_urls.columns=header_names
                    df_urls=pd.concat([df_urls, pd.DataFrame([header_row], columns=header_names)])                
                else: #event it is not, still assign the header_name
                    df_urls.columns=header_names 
                
                return dict(zip(df_urls['url'], df_urls['status_code'])) #format: url:status_code. i.e., url is the key
                #another options is: df_urls.to_dict('list') #format of {url:[url1, url2...], status_code:[0, 200...]}
            else:
                return {}        

   

In [52]:
colleges = [["UCLA", "http://www.ucla.edu", []],
           ["Stanford", "https://www.stanford.edu/", []],
           ["Yale", "https://www.yale.edu/", []],
           ["UW", "http://www.washington.edu/", []]]
for college in colleges:
    cr = CollegeCrawl(college[0], college[1], college[2])
    cr.crawl()


There was an error
************Crawl UCLA(http://www.ucla.edu)************
Total number of URLs visisted:  6
Total number of URLs rejected:  0
URLs visited (ranked with priority words):
 ['http://www.ucla.edu/research/research-across-campus', 'http://www.ucla.edu/campus-life', 'http://www.ucla.edu/optimists/fisher', 'http://www.ucla.edu/about/iconic-and-influential-alumni', 'http://www.ucla.edu/academics', 'http://www.ucla.edu']
************Crawl Stanford(https://www.stanford.edu/)************
Total number of URLs visisted:  6
Total number of URLs rejected:  0
URLs visited (ranked with priority words):
 ['https://www.stanford.edu/#b', 'https://www.stanford.edu/#y', 'https://www.stanford.edu/atoz', 'https://www.stanford.edu/#h', 'https://www.stanford.edu/#v', 'https://www.stanford.edu/']
************Crawl Yale(https://www.yale.edu/)************
Total number of URLs visisted:  6
Total number of URLs rejected:  0
URLs visited (ranked with priority words):
 ['https://www.yale.edu/life-yale

In [ ]:
def getResponse(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url)
    #response = requests.get(url,None, headers=headers)
    print("response status code for URL {} is {}".format(url,response.status_code))
    print("request headers for URL {} is {}".format(url, response.request.headers))
    
    
    #help(response)
    #help(requests.get)
    
    print(response.text)
 


getResponse("https://www.mit.edu")
#getResponseCode("http://www.stanford.edu")
#getResponseCode("http://www.harvard.edu")

In [ ]:
%debug

In [ ]:
_ih[-5:]